In [7]:
from scvi.hub import HubModel
from botocore.config import Config
import botocore

model = HubModel.pull_from_s3(
    s3_bucket="cellxgene-contrib-public",
    s3_path="models/scvi/2024-02-12/homo_sapiens/modelhub",
    pull_anndata=False,
    config=Config(signature_version=botocore.UNSIGNED),
)

In [19]:
import anndata

# load training data
adata = anndata.read_h5ad("../data/RNA/raw/train/cellxgene_pseudo_bulk_3_5k.h5ad")

In [22]:
adata.var_names

Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167',
       ...
       'ENSG00000288638', 'ENSG00000288642', 'ENSG00000288649',
       'ENSG00000288675', 'ENSG00000288677', 'ENSG00000288701',
       'ENSG00000288702', 'ENSG00000288705', 'ENSG00000288709',
       'ENSG00000288722'],
      dtype='object', length=38030)

In [20]:
import scanpy as sc

# Calculate highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=8000, flavor="seurat_v3")
# Subset to highly variable genes
adata_train = adata[:, adata.var.highly_variable]

KeyboardInterrupt: 

In [21]:
import os
import tempfile

save_dir = tempfile.TemporaryDirectory()
adata_path = os.path.join(save_dir.name, "adata-spinal-cord-minified.h5ad")

adata_og = sc.read(
    adata_path,
    backup_url="https://cellxgene-contrib-public.s3.amazonaws.com/models/scvi/2024-02-12/homo_sapiens/adata-spinal-cord-minified.h5ad",
)

  0%|          | 0.00/45.1M [00:00<?, ?B/s]

In [24]:
# Get intersection of var_names between the two datasets
common_genes = list(set(adata.var_names).intersection(set(adata_og.var_names)))
print(f"Number of common genes: {len(common_genes)}")

Number of common genes: 5619


In [16]:
model.load_model(adata=adata.copy())

INFO     Loading model...                                                                                          
INFO     File /tmp/tmpiob9kf8k/model.pt already downloaded                                                         


/h/menger/git/adata_hf_datasets/.venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /h/menger/git/adata_hf_datasets/.venv/lib/python3.10 ...
/h/menger/git/adata_hf_datasets/.venv/lib/python3.10/site-packages/scvi/model/base/_base_model.py:713: UserWarning: `var_names` for the loaded `adata` does not match those of the `adata` used to train the model. For valid results, the former should match the latter.
  _validate_var_names(adata, var_names)


TypeError: '<' not supported between instances of 'float' and 'str'